<a href="https://colab.research.google.com/github/fxr1115/Learning/blob/main/Python3-Algorithm/array-string/2_2D_array.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from typing import List

### 二维数组

- 是一种结构较为特殊的数组，将数组中的每个元素变成了一维数组
- 类似于一维数组，对于一个二元数组，计算机同样在内存中申请一段**连续**的空间，并记录第一行数组的索引位置，即**`A[0][0]**`的内存地址

### 题目1 旋转矩阵
一幅由$N \times N$矩阵表示的图像，其中每个像素的大小为4字节，设计一种算法，将图像旋转90度
- **不占用额外内存空间**

In [7]:
def rotate(matrix: List[List[int]]) -> None:
    """
    Do not return anything, modify matrix in-place instead.
    """
    n = len(matrix)
    col = [[] for _ in range(n)]
    for j in range(n):
        for i in range(n - 1, -1, -1):
            col[j].append(matrix[i][j])
    for k in range(n):
        matrix[k] = col[k]

In [8]:
rotate([[1,2,3],[4,5,6],[7,8,9]])

**代码问题**：（额外空间问题，操作复杂度问题）
- 创建了额外的`col`二维列表，占用了额外的空间
- 通过多次嵌套循环和赋值操作，可以进一步优化

**优化方法**：
- 使用原地操作，避免创建额外的列表
- 步骤分解：转置矩阵（行列互换）+ 反转每一行（每一行元素反转）

- 多变量赋值，一次性完成`matrix[i][j], matrix[j][i]`的交换
- `list.reverse()`是列表的内置方法，**原地反转**列表中的元素顺序

In [9]:
def rotate(matrix: List[List[int]]) -> None:
    """
    Do not return anything, modify matrix in-place instead.
    """
    n = len(matrix)
    for i in range(n):
        for j in range(i, n):
            matrix[i][j], matrix[j][i] = matrix[j][i], matrix[i][j]

    for i in range(n):
        matrix[i].reverse()

**使用深拷贝**，修改任何部分都不会影响原对象
- 但是不符合题目要求

In [10]:
import copy

def rotate(matrix: List[List[int]]) -> None:
    """
    Do not return anything, modify matrix in-place instead.
    """
    matrix1 = copy.deepcopy(matrix)
    row = len(matrix)
    col = len(matrix[0])
    for i in range(row):
        for j in range(col):
            matrix[j][row - i - 1] = matrix1[i][j]

In [11]:
rotate([[1,2,3],[4,5,6],[7,8,9]])

### 题目2 零矩阵
若$M \times N$矩阵中某个元素为0，则将其所在的行和列清零

下面是**错的**

In [20]:
def setZeroes(matrix: List[List[int]]) -> None:
    """
    Do not return anything, modify matrix in-place instead.
    """
    m = len(matrix)
    n = len(matrix[0])
    matrix1 = copy.deepcopy(matrix)
    for i in range(m):
        for j in range(n):
            if matrix1[i][j] == 0:
                matrix[i] = [0] * m
                for i in range(m):
                    matrix[i][j] = 0
    return matrix

In [22]:
setZeroes([[0,1,2,0],[3,4,5,2],[1,3,1,5]])

[[0, 0, 0], [0, 4, 5, 2], [0, 3, 1, 5]]

**代码问题**：（深拷贝没有解决问题，逻辑错误<行和列清零顺序冲突>）
- `matrix[i] = [0] * m` 将整个行替换为一个新列表，会**破坏原矩阵结构**，导致列清零逻辑错误
    - 第i行原始引用被替换，`matrix[i]`不再与矩阵中其他部分共享结构，失去与原始矩阵的关联
    - 在清零过程中不断修改`matrix`，导致下一步清零的逻辑依赖了已经被改变的矩阵，产生错误
- 代码逻辑未能正确区分哪些行和列需要清零，清零操作互相干扰

In [23]:
def setZeroes(matrix: List[List[int]]) -> None:
    """
    Do not return anything, modify matrix in-place instead.
    """
    m = len(matrix)
    n = len(matrix[0])
    matrix1 = copy.deepcopy(matrix)
    for i in range(m):
        for j in range(n):
            if matrix1[i][j] == 0:
                for k in range(n):
                    matrix[i][k] = 0
                for k in range(m):
                    matrix[k][j] = 0
    return matrix

In [24]:
setZeroes([[0,1,2,0],[3,4,5,2],[1,3,1,5]])

[[0, 0, 0, 0], [0, 4, 5, 0], [0, 3, 1, 0]]

**优化**
- 使用**标记法**，记录需要清零的行和列
- 避免使用深拷贝
- 使用原地操作

- `rows`和`cols`也可以使用列表来表示，添加时使用`.append()`即可

In [27]:
def setZeroes(matrix: List[List[int]]) -> None:
    """
    Do not return anything, modify matrix in-place instead.
    """
    m, n = len(matrix), len(matrix[0])
    row, col = [], []

    # 标记需要清零的行和列
    for i in range(m):
        for j in range(n):
            if matrix[i][j] == 0:
                row.append(i)
                col.append(j)

    # 根据标记清零行
    for i in row:
        for j in range(n):
            matrix[i][j] = 0

    for j in col:
        for i in range(m):
            matrix[i][j] = 0

    return matrix

In [28]:
setZeroes([[0,1,2,0],[3,4,5,2],[1,3,1,5]])

[[0, 0, 0, 0], [0, 4, 5, 0], [0, 3, 1, 0]]

### 题目3 对角线遍历
大小为$M \times N$的矩阵`mat`，以对角线遍历的顺序，用一个数组返回这个矩阵中的所有元素

**不会**：如何表示索引

In [ ]:
class Solution:
    def findDiagonalOrder(self, mat: List[List[int]]) -> List[int]:
        arr = []
        # 行列和是常数
        n = len(mat) + len(mat[0]) - 1
        for i in range(n):
            if i % 2 == 0:  # 从左下到右上
                for j in range(i + 1):
                    arr.append(mat[j][i - j])
            else:
                for j in range(i + 1):
                    arr.append(mat[i - j][j])
        return arr

**代码问题**：
- 没有区分矩阵上下三角形部分：上三角部分，`i`的范围受列的限制

**优化思路**：
- 明确对角线遍历的范围（这个思考了一半）
- 区分遍历方向，分奇偶（这个也想到了）
- 利用`max`和`min`限制索引范围（这个没有想到）
- 先得到对角线元素，再分奇偶

In [29]:
class Solution:
    def findDiagonalOrder(self, mat: List[List[int]]) -> List[int]:
        arr = []
        m, n = len(mat), len(mat[0])
        for i in range(m + n - 1):
            # 定义当前对角线的起点（行和列）
            if i < m:  # 上三角区域
                row, col = i, 0
            else:
                row, col = m - 1, i - m + 1

            diagonal = []

            # 遍历当前对角线
            while row >= 0 and col < n:
                diagonal.append(mat[row][col])
                row -= 1
                col += 1

            # 根据对角线编号的奇偶确定遍历方向
            if i % 2 == 0:
                arr.extend(diagonal)
            else:
                arr.extend(diagonal[::-1])
        return arr

In [30]:
Solution().findDiagonalOrder([[1,2,3],[4,5,6],[7,8,9]])

[1, 2, 4, 7, 5, 3, 6, 8, 9]

**直接操作结果列表，索引计算更加简洁，更高效的边界限制逻辑**

In [31]:
class Solution:
    def findDiagonalOrder(self, mat: List[List[int]]) -> List[int]:
        m, n = len(mat), len(mat[0])
        result = [mat[0][0]]
        for d in range(1, m + n - 1):
            if d % 2 == 0:
                x = d if d < m else m - 1
                y = 0 if d < m else d - m + 1
                while x >= 0 and y < n:
                    result.append(mat[x][y])
                    x -= 1
                    y += 1

            else:
                y = d if d < n else n - 1
                x = 0 if d < n else d - n + 1
                while x < m and y >= 0:
                    result.append(mat[x][y])
                    x += 1
                    y -= 1
        return result

In [32]:
Solution().findDiagonalOrder([[1,2,3],[4,5,6],[7,8,9]])

[1, 2, 4, 7, 5, 3, 6, 8, 9]